<a href="https://colab.research.google.com/github/tanvir-ahammedd/llm-finetuning-lab/blob/main/Qwen2_5_Coder_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers datasets -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s 

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-7B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("Qwen 2.5 Coder 7B loaded successfully!")
print("This model is specifically optimized for coding tasks!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.3: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Qwen 2.5 Coder 7B loaded successfully!
This model is specifically optimized for coding tasks!


In [ ]:
FastLanguageModel.for_inference(model)

# Qwen uses ChatML format, but we'll use Alpaca for consistency
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

instruction = "Write a Python function to calculate the factorial of a number."
input_text = ""

inputs = tokenizer([alpaca_prompt.format(instruction, input_text)], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
response = tokenizer.batch_decode(outputs)[0]
print("\nPre-trained model output:\n", response)


Pre-trained model output:
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a Python function to calculate the factorial of a number.

### Response:
```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
```<|endoftext|>


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    max_seq_length = max_seq_length,
)

print("LoRA adapters configured for Qwen 2.5 Coder!")

Unsloth 2025.10.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


LoRA adapters configured for Qwen 2.5 Coder!


In [ ]:
from google.colab import files
uploaded = files.upload()

from datasets import load_dataset

dataset = load_dataset("csv", data_files="my_dataset.csv")["train"]

print("Example raw row:")
print(dataset[0])

Saving my_dataset.csv to my_dataset.csv


Generating train split: 0 examples [00:00, ? examples/s]

Example raw row:
{'id': 1, 'instruction': 'স্ট্রিং থেকে প্রদত্ত অক্ষরের প্রথম এবং শেষ উপস্থিতি অপসারণের জন্য একটি পাইথন ফাংশন লিখুন।', 'response': 'def remove_Occ(s,ch): \r\n    for i in range(len(s)): \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    for i in range(len(s) - 1,-1,-1):  \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    return s ', 'test_list': '"[\'assert remove_Occ(\\"hello\\",\\"l\\") == \\"heo\\"\', \'assert remove_Occ(\\"abcda\\",\\"a\\") == \\"bcd\\"\', \'assert remove_Occ(\\"PHP\\",\\"P\\") == \\"H\\"\']"'}


In [ ]:
alpaca_prompt = """Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
{}

### Example Tests:
{}

### Response:
{}"""

def format_input_prompt(examples):
    instructions = examples["instruction"]
    test_lists = examples["test_list"]
    outputs = examples["response"]

    texts = []
    for instruction, test_list, output in zip(instructions, test_lists, outputs):
        text = alpaca_prompt.format(instruction, test_list, output) + tokenizer.eos_token
        texts.append(text)

    return {"text": texts}

dataset = dataset.map(format_input_prompt, batched=True)

print("\nExample formatted text:\n")
print(dataset["text"][0])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]


Example formatted text:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
স্ট্রিং থেকে প্রদত্ত অক্ষরের প্রথম এবং শেষ উপস্থিতি অপসারণের জন্য একটি পাইথন ফাংশন লিখুন।

### Example Tests:
"['assert remove_Occ(\"hello\",\"l\") == \"heo\"', 'assert remove_Occ(\"abcda\",\"a\") == \"bcd\"', 'assert remove_Occ(\"PHP\",\"P\") == \"H\"']"

### Response:
def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s <|endoftext|>


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1234,
        output_dir = "outputs",
        report_to = "none",
        save_strategy = "steps",
        save_steps = 50,
    ),
)

print("\nStarting training with Qwen 2.5 Coder 7B...")
print("Qwen Coder is already optimized for programming tasks!")
Trainer_Stats = trainer.train()
print("\nTraining complete!")
Trainer_Stats

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.



Starting training with Qwen 2.5 Coder 7B...
Qwen Coder is already optimized for programming tasks!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.180900
2,1.329500
3,1.232100
4,1.330500
5,1.040900
6,0.987200
7,0.975000
8,0.843900
9,0.819700
10,0.803400



Training complete!


TrainOutput(global_step=60, training_loss=0.5649338920911153, metrics={'train_runtime': 493.6084, 'train_samples_per_second': 0.972, 'train_steps_per_second': 0.122, 'total_flos': 6695565316392960.0, 'train_loss': 0.5649338920911153, 'epoch': 0.96})

In [ ]:
model.save_pretrained("./qwen_coder_lora")
tokenizer.save_pretrained("./qwen_coder_lora")

('./qwen_coder_lora/tokenizer_config.json',
 './qwen_coder_lora/special_tokens_map.json',
 './qwen_coder_lora/vocab.json',
 './qwen_coder_lora/merges.txt',
 './qwen_coder_lora/added_tokens.json',
 './qwen_coder_lora/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

test_instruction = "একটি নির্দিষ্ট পূর্ণসংখ্যা অ্যারেতে কোনো সদৃশ উপাদান আছে কিনা তা খুঁজে বের করার জন্য একটি ফাংশন লিখুন।"
test_input = "[]"

inputs = tokenizer([alpaca_prompt.format(test_instruction, test_input, "")], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("\n Fine-tuned model output:\n")
print(response)


 Fine-tuned model output:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
একটি নির্দিষ্ট পূর্ণসংখ্যা অ্যারেতে কোনো সদৃশ উপাদান আছে কিনা তা খুঁজে বের করার জন্য একটি ফাংশন লিখুন।

### Example Tests:
[]

### Response:
def find_duplicate(arr):
    for i in range(len(arr)):
        if arr[abs(arr[i])] >= 0:
            arr[abs(arr[i])] = -arr[abs(arr[i])]
        else:
            return abs(arr[i])
    return -1


In [ ]:
FastLanguageModel.for_inference(model)

test_instruction = "একটি ফাংশন লিখুন যাতে সবচেয়ে ছোট স্ট্রিং এর দৈর্ঘ্য খুঁজে পাওয়া যায় যার মধ্যে str1 এবং str2 উভয়ই সাবসেকভেন্স হিসাবে থাকে।"
test_input = "[]"

inputs = tokenizer([alpaca_prompt.format(test_instruction, test_input, "")], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("\n Fine-tuned model output:\n")
print(response)


 Fine-tuned model output:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
একটি ফাংশন লিখুন যাতে সবচেয়ে ছোট স্ট্রিং এর দৈর্ঘ্য খুঁজে পাওয়া যায় যার মধ্যে str1 এবং str2 উভয়ই সাবসেকভেন্স হিসাবে থাকে।

### Example Tests:
[]

### Response:
def shortest_common_supersequence(str1, str2):
    m = len(str1)
    n = len(str2)
    dp = [[0] * (n + 1) for i in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif str1[i - 1] == str2[j - 1]:
                dp[i][j] = 1 + dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 +


In [ ]:
FastLanguageModel.for_inference(model)

test_instruction = " বেজোড় সংখ্যক বার উপস্থিত উপাদানটি খুঁজে পেতে একটি পাইথন ফাংশন লিখুন।"
test_input = "[]"

inputs = tokenizer([alpaca_prompt.format(test_instruction, test_input, "")], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("\n Fine-tuned model output:\n")
print(response)


 Fine-tuned model output:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
 বেজোড় সংখ্যক বার উপস্থিত উপাদানটি খুঁজে পেতে একটি পাইথন ফাংশন লিখুন।

### Example Tests:
[]

### Response:
def count_odd_frequency(arr):
    freq = {}
    for i in arr:
        if i in freq:
            freq[i] += 1
        else:
            freq[i] = 1
    count = 0
    for i in freq:
        if freq[i] % 2 != 0:
            count += 1
    return count


In [ ]:
FastLanguageModel.for_inference(model)

test_instruction = "একটি ফাংশন লিখুন যাতে দুইটি সংখ্যার সর্বোচ্চ সংখ্যা পাওয়া যায়।"
test_input = "[]"

inputs = tokenizer([alpaca_prompt.format(test_instruction, test_input, "")], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("\n Fine-tuned model output:\n")
print(response)


 Fine-tuned model output:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
একটি ফাংশন লিখুন যাতে দুইটি সংখ্যার সর্বোচ্চ সংখ্যা পাওয়া যায়।

### Example Tests:
[]

### Response:
def max_of_two(x, y):
    if x > y:
        return x
    return y


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test_dataset.csv to test_dataset.csv


In [ ]:
import pandas as pd
test_df = pd.read_csv("test_dataset.csv")
print("Total test prompts:", len(test_df))
test_df.head()


Total test prompts: 30


,id,instruction,test_list
0,1,স্ট্রিং থেকে প্রদত্ত অক্ষরের প্রথম এবং শেষ উপস...,"['assert remove_Occ(\""hello\"",\""l\"") == \""heo\""']"
1,2,একটি প্রদত্ত ম্যাট্রিক্সকে তার সারিগুলির যোগফল...,"['assert sort_matrix([[1, 2, 3], [2, 4, 5], [1..."
2,3,একটি ফাংশন লিখুন যা একটি অভিধানে সবচেয়ে সাধার...,"['assert count_common([\'red\',\'green\',\'bla..."
3,4,একটি ত্রিভুজাকার প্রিজমের আয়তন খুঁজে বের করার...,"['assert find_Volume(10,8,6) == 240']"
4,5,একটি স্ট্রিংকে ছোট অক্ষরে বিভক্ত করার জন্য একট...,"['assert split_lowerstring(\""AbCd\"")==[\'bC\',..."


In [ ]:
import torch
from tqdm import tqdm

FastLanguageModel.for_inference(model)

results = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    instruction = str(row["instruction"])
    test_list = str(row["test_list"])

    prompt = alpaca_prompt.format(instruction, test_list, "")

    inputs = tokenizer([prompt], return_tensors='pt').to('cuda')

    outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    results.append({
        "id": row["id"],
        "instruction": instruction,
        "test_list": test_list,
        "generated_response": response
    })


100%|██████████| 30/30 [01:29<00:00,  2.99s/it]


In [ ]:
result_df = pd.DataFrame(results)
result_df.to_csv("fine_tuned_test_results.csv", index=False)
print("All test prompts processed and saved as fine_tuned_test_results.csv")

All test prompts processed and saved as fine_tuned_test_results.csv


In [ ]:
from google.colab import files
files.download("fine_tuned_test_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r qwen_coder_lora /content/drive/MyDrive/qwen_coder_lora

Mounted at /content/drive


SyntaxError: invalid syntax (ipython-input-3608887273.py, line 1)

In [ ]:
!ls /content/drive/MyDrive

 221002036_KSA_2.pdf
 221002036_microct.pdf
'221002036(Tanvir_Ahammed)[1] (1).pdf'
'221002036(Tanvir_Ahammed)[1].pdf'
'(221002036) Tanvir Ahammed.gslides'
'221D9 (1).xlsm.gsheet'
'221D9 (2).xlsm.gsheet'
'221D9 (3).xlsm.gsheet'
 221D9.xlsm.gsheet
 2pmq7j1593422273.pdf
 App
'Assignment cover page (1).gdoc'
'Assignment cover page.gdoc'
'Assignment on Relation.pdf'
'Book1 (1).gsheet'
 Classroom
'coinome wallet recovery phrase'
'Colab Notebooks'
'Copy of 82325-Internet of things ppt presentation free download .gslides'
'Copy of Atomic structure-221013.pdf'
'Copy of Minimalist Business Slides XL by Slidesgo.gslides'
'Copy of video1221698814.mp4'
'CSE_301_Lecture-1 Introduction_copy.pdf'
'Dawn of Programming Contest (March 2015).pdf'
'DBS LR2- Tanvir (1).gdoc'
'DBS LR2- Tanvir (2).gdoc'
'DBS LR2- Tanvir.gdoc'
'Document 3.pdf'
'doge coin wallet key'
 Electroneum_2.3.4.apk
 Etherum
'Ethoreum address '
'EXP-6_221002036 (1).docx'
 EXP-6_221002036.docx
'Freelancing '
'Functional Units of Digital S

In [ ]:
!ls /content/drive/MyDrive/qwen_coder_lora

adapter_config.json	   merges.txt		    tokenizer_config.json
adapter_model.safetensors  README.md		    tokenizer.json
added_tokens.json	   special_tokens_map.json  vocab.json


In [ ]:
!zip -r qwen_coder_lora.zip qwen_coder_lora
from google.colab import files
files.download("qwen_coder_lora.zip")

	zip warning: name not matched: qwen_coder_lora

zip error: Nothing to do! (try: zip -r qwen_coder_lora.zip . -i qwen_coder_lora)


FileNotFoundError: Cannot find file: qwen_coder_lora.zip